In [1]:

import os
import re
import numpy as np

from utils import access_raster, write_single_channel_gtiff

import rasterio as rio
from rasterio.windows import get_data_window, transform, Window

from shapely.geometry import box
from shapely.geometry import mapping

import geopandas as gpd
import rioxarray as rxr

In [2]:
unitato_res = "/home/abetatos/Escritorio/TFM_Dataset/2.Dataset_res5/"
unitato_base = "/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/"

In [3]:
input_path_HS = os.path.join(unitato_res, "HS")
output_path_HS = os.path.join(unitato_base, "HS")
os.makedirs(output_path_HS, exist_ok=True)

HS_files = [f for f in os.listdir(input_path_HS) if re.search(".tiff?$", f)]

# HS

In [5]:

for f in HS_files: 
    print(f)
    # Apply window to get short 
    with rio.open(os.path.join(input_path_HS, f)) as src:
        profile = src.profile.copy()
        data_window = get_data_window(src.read(masked=True))
        data_transform = transform(data_window, src.transform)
        profile.update(
            transform=data_transform,
            height=data_window.height,
            width=data_window.width
            )
        data = src.read(window=data_window)
        array = data[0]
        array[array<=profile['nodata']] = -9999.
        profile.update({"nodata": -9999.})

    out_path = os.path.join(output_path_HS,  os.path.splitext(f)[0])
    try: 
        os.makedirs(out_path)
    except: 
        pass
    
    with rio.open(os.path.join(out_path, "HS.tif"), "w", **profile) as src: 
        src.write(data)

    HS_raster = access_raster(os.path.join(out_path, "HS.tif"))
    HS_geodf = gpd.GeoDataFrame(geometry=[box(*HS_raster['extent'], ccw=False)], crs=HS_raster['meta']['crs'])
    HS_geodf
    
    out_extent = os.path.join(out_path, "extent.shp")
    HS_geodf.to_file(out_extent)

snowmap_20160309_ads_100_extent_maskiert_nosnow0_noausreisser.tif
14.999023 -7098.0605 -9999.0
HS_Davos_20211604_2m_epsg2056.tif
9.91038 -6012.1206 -9999.0
snowmap_2012_ads_80_extent_maskiert_nosnow0_noausreisser.tif
14.994385 -6836.6567 -9999.0
HS_Laucherenalp_20220224_2m_epsg2056.tif
26.533447 -9174.224 -9999.0
snowmap_2010_ads_80_extent_maskiert_nosnow0_noausreisser.tif
15.0 -6921.3945 -9999.0
HS_Davos_20200406_2m_epsg2056.tif
9.553748 -6051.4487 -9999.0
HS_c_20220223_1m_epsg2056.tif
16.229492 -4975.13 -9999.0
snowmap_20160420_ads_100_extent_maskiert_nosnow0_noausreisser.tif
14.998779 -7095.5176 -9999.0
snowmap_2014_ads_100_extent_maskiert_nosnow0_noausreisser.tif
14.975586 -7146.1094 -9999.0
HS_Davos_20180411_2m_epsg2056.tif
9.043191 -6762.6606 -9999.0
snowmap_2013_ads_80_extent_maskiert_nosnow0_noausreisser.tif
14.994385 -6841.78 -9999.0
HS_Davos_20170316_2m_epsg2056.tif
9.911215 -5579.924 -9999.0
HS_Laucherenalp_20220511_1m_epsg2056.tif
9.746826 -7623.67 -9999.0
HS_Saflischpass_2

DEM

In [42]:
input_path_DEM = os.path.join(unitato_res, "DEM")
output_path_DEM = os.path.join(unitato_base, "DEM")

input_path_HS = rf"{unitato_base}HS"
for f in HS_files: 
    print(f)
    for DEM_file in os.listdir(input_path_DEM): 
        input_DEM = os.path.join(input_path_DEM, DEM_file)
        aoi = os.path.join(input_path_HS, os.path.splitext(f)[0], "extent.shp")
        try:
            cropped_im_data = access_raster(input_DEM, aoi=aoi)
        except ValueError: 
            continue
        print("\t", DEM_file)
        
        gdf = gpd.GeoDataFrame.from_file(aoi)
        rxr_DEM = rxr.open_rasterio(input_DEM, masked=True).squeeze()
        rxr_DEM = rxr_DEM.rio.clip(gdf.geometry.apply(mapping))
        
        try:
            os.makedirs(os.path.join(output_path_DEM, os.path.splitext(f)[0]))
        except: 
            pass
        output_file = os.path.join(output_path_DEM, os.path.splitext(f)[0], "DEM.tif")
        rxr_DEM.rio.to_raster(output_file)

        with rio.open(output_file, "r") as src: 
            array = src.read()
            nodata = src.profile['nodata']
            print((array==nodata).sum(), (array!=nodata).sum())

snowmap_20160309_ads_100_extent_maskiert_nosnow0_noausreisser.tif


/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


0 15104440
HS_Davos_20211604_2m_epsg2056.tif


/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


0 14927616
snowmap_2012_ads_80_extent_maskiert_nosnow0_noausreisser.tif


...

/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


0 511224
snowmap_20160126_ads_100_extent_maskiert_nosnow0_noausreisser.tif


/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


0 15168156
snowmap_2015_ads_100_extent_maskiert_nosnow0_noausreisser.tif
0 15219900


/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


# SCE

In [5]:
input_path_SCE = os.path.join(unitato_res, "SCE")
output_path_SCE = os.path.join(unitato_base, "SCE")

input_path_HS = rf"{unitato_base}HS"

input_SCE = f"{unitato_res}SCE/SCE.tif"

# rxr_DEM = rxr.open_rasterio(input_SCE, masked=True).squeeze()
    
    
for f in HS_files: 
    print(f)
    aoi = os.path.join(input_path_HS, os.path.splitext(f)[0], "extent.shp")
    gdf = gpd.GeoDataFrame.from_file(aoi)
    
    cropped_im_data = access_raster(input_SCE, aoi=aoi)
    
    try:
        os.makedirs(os.path.join(output_path_SCE, os.path.splitext(f)[0]))
    except: 
        pass
    
    write_single_channel_gtiff(cropped_im_data['array'].squeeze(), 
                                transform=cropped_im_data['transform'], 
                                meta = cropped_im_data['meta'],  
                                out_path = os.path.join(output_path_SCE, os.path.splitext(f)[0], "SCE.tif"), 
                                nodata = cropped_im_data['meta']['nodata']
                                )

snowmap_20160309_ads_100_extent_maskiert_nosnow0_noausreisser.tif
HS_Davos_20211604_2m_epsg2056.tif
snowmap_2012_ads_80_extent_maskiert_nosnow0_noausreisser.tif
HS_Laucherenalp_20220224_2m_epsg2056.tif
snowmap_2010_ads_80_extent_maskiert_nosnow0_noausreisser.tif
HS_Davos_20200406_2m_epsg2056.tif
HS_c_20220223_1m_epsg2056.tif
snowmap_20160420_ads_100_extent_maskiert_nosnow0_noausreisser.tif
snowmap_2014_ads_100_extent_maskiert_nosnow0_noausreisser.tif
HS_Davos_20180411_2m_epsg2056.tif
snowmap_2013_ads_80_extent_maskiert_nosnow0_noausreisser.tif
HS_Davos_20170316_2m_epsg2056.tif
HS_Laucherenalp_20220511_1m_epsg2056.tif
HS_Saflischpass_20220512_1m_epsg2056.tif
snowmap_20160126_ads_100_extent_maskiert_nosnow0_noausreisser.tif
snowmap_2015_ads_100_extent_maskiert_nosnow0_noausreisser.tif


# Aspect

In [37]:
input_path_Aspect = os.path.join(unitato_res, "Aspect")
output_path_Aspect = os.path.join(unitato_base, "Aspect")

input_path_HS = rf"{unitato_base}HS"

for f in HS_files: 
    print(f)
    for Aspect_file in os.listdir(input_path_Aspect): 
        input_Aspect = os.path.join(input_path_Aspect, Aspect_file)
        aoi = os.path.join(input_path_HS, os.path.splitext(f)[0], "extent.shp")
        try:
            cropped_im_data = access_raster(input_Aspect, aoi=aoi)
        except ValueError: 
            continue
        print("\t", Aspect_file)
        print(input_Aspect)
        print(aoi)
        
        gdf = gpd.GeoDataFrame.from_file(aoi)
        rxr_Aspect = rxr.open_rasterio(input_Aspect, masked=True).squeeze()
        rxr_Aspect = rxr_Aspect.rio.clip(gdf.geometry.apply(mapping))
        
        try:
            os.makedirs(os.path.join(output_path_Aspect, os.path.splitext(f)[0]))
        except: 
            pass
        output_file = os.path.join(output_path_Aspect, os.path.splitext(f)[0], "Aspect.tif")
        rxr_Aspect.rio.to_raster(output_file)

snowmap_20160309_ads_100_extent_maskiert_nosnow0_noausreisser.tif
	 Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/2.Dataset_res5/Aspect/Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/HS/snowmap_20160309_ads_100_extent_maskiert_nosnow0_noausreisser/extent.shp


/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


0
HS_Davos_20211604_2m_epsg2056.tif
	 Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/2.Dataset_res5/Aspect/Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/HS/HS_Davos_20211604_2m_epsg2056/extent.shp


/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


0
snowmap_2012_ads_80_extent_maskiert_nosnow0_noausreisser.tif
	 Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/2.Dataset_res5/Aspect/Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/HS/snowmap_2012_ads_80_extent_maskiert_nosnow0_noausreisser/extent.shp


...

/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


0
snowmap_20160126_ads_100_extent_maskiert_nosnow0_noausreisser.tif
	 Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/2.Dataset_res5/Aspect/Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/HS/snowmap_20160126_ads_100_extent_maskiert_nosnow0_noausreisser/extent.shp


/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


0
snowmap_2015_ads_100_extent_maskiert_nosnow0_noausreisser.tif
	 Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/2.Dataset_res5/Aspect/Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/HS/snowmap_2015_ads_100_extent_maskiert_nosnow0_noausreisser/extent.shp
0


/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


# Slope

In [16]:
input_path_Slope = os.path.join(unitato_res, "Slope")
output_path_Slope = os.path.join(unitato_base, "Slope")

input_path_HS = rf"{unitato_base}HS"

for f in HS_files: 
    print(f)
    for Slope_file in os.listdir(input_path_Slope): 
        input_Slope = os.path.join(input_path_Slope, Slope_file)
        aoi = os.path.join(input_path_HS, os.path.splitext(f)[0], "extent.shp")
        try:
            cropped_im_data = access_raster(input_Slope, aoi=aoi)
        except ValueError: 
            continue
        print("\t", Slope_file)
        print(input_Slope)
        print(aoi)
        
        gdf = gpd.GeoDataFrame.from_file(aoi)
        rxr_Slope = rxr.open_rasterio(input_Slope, masked=True).squeeze()
        rxr_Slope = rxr_Slope.rio.clip(gdf.geometry.apply(mapping))
        
        try:
            os.makedirs(os.path.join(output_path_Slope, os.path.splitext(f)[0]))
        except: 
            pass
        rxr_Slope.rio.to_raster(os.path.join(output_path_Slope, os.path.splitext(f)[0], "Slope.tif"))

snowmap_20160309_ads_100_extent_maskiert_nosnow0_noausreisser.tif
	 Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/2.Dataset_res5/Slope/Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/HS/snowmap_20160309_ads_100_extent_maskiert_nosnow0_noausreisser/extent.shp


/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


HS_Davos_20211604_2m_epsg2056.tif
	 Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/2.Dataset_res5/Slope/Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/HS/HS_Davos_20211604_2m_epsg2056/extent.shp


/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


snowmap_2012_ads_80_extent_maskiert_nosnow0_noausreisser.tif
	 Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/2.Dataset_res5/Slope/Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/HS/snowmap_2012_ads_80_extent_maskiert_nosnow0_noausreisser/extent.shp


...

/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


snowmap_20160126_ads_100_extent_maskiert_nosnow0_noausreisser.tif
	 Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/2.Dataset_res5/Slope/Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/HS/snowmap_20160126_ads_100_extent_maskiert_nosnow0_noausreisser/extent.shp


/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


snowmap_2015_ads_100_extent_maskiert_nosnow0_noausreisser.tif
	 Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/2.Dataset_res5/Slope/Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/HS/snowmap_2015_ads_100_extent_maskiert_nosnow0_noausreisser/extent.shp


/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


# TPI

In [21]:
for filter_size in [11]: #[7, 11, 22, 50]:

    input_path_tpi = os.path.join(unitato_res, f"TPI")
    output_path_tpi = os.path.join(unitato_base, f"TPI")
    print("TPI:", filter_size)
    
    input_path_HS = f"{unitato_base}HS"
    for f in HS_files: 
        print(f)
        for tpi_file in os.listdir(input_path_tpi): 
            input_tpi = os.path.join(input_path_tpi, tpi_file)
            aoi = os.path.join(input_path_HS, os.path.splitext(f)[0], "extent.shp")
            try:
                cropped_im_data = access_raster(input_tpi, aoi=aoi)
            except ValueError as e: 
                print(e)
                continue
            print("\t", tpi_file)
            print(input_tpi)
            print(aoi)
            
            gdf = gpd.GeoDataFrame.from_file(aoi)
            rxr_tpi = rxr.open_rasterio(input_tpi, masked=True).squeeze()
            rxr_tpi = rxr_tpi.rio.clip(gdf.geometry.apply(mapping))
            
            try:
                os.makedirs(os.path.join(output_path_tpi, os.path.splitext(f)[0]))
            except: 
                pass
            rxr_tpi.rio.to_raster(os.path.join(output_path_tpi, os.path.splitext(f)[0], "TPI.tif"))

TPI: 11
snowmap_20160309_ads_100_extent_maskiert_nosnow0_noausreisser.tif
	 Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/2.Dataset_res5/TPI/Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/HS/snowmap_20160309_ads_100_extent_maskiert_nosnow0_noausreisser/extent.shp


/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


Input shapes do not overlap raster.
HS_Davos_20211604_2m_epsg2056.tif
	 Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/2.Dataset_res5/TPI/Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/HS/HS_Davos_20211604_2m_epsg2056/extent.shp


/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


Input shapes do not overlap raster.
snowmap_2012_ads_80_extent_maskiert_nosnow0_noausreisser.tif
	 Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/2.Dataset_res5/TPI/Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/HS/snowmap_2012_ads_80_extent_maskiert_nosnow0_noausreisser/extent.shp


...

/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


snowmap_20160126_ads_100_extent_maskiert_nosnow0_noausreisser.tif
	 Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/2.Dataset_res5/TPI/Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/HS/snowmap_20160126_ads_100_extent_maskiert_nosnow0_noausreisser/extent.shp


/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


Input shapes do not overlap raster.
snowmap_2015_ads_100_extent_maskiert_nosnow0_noausreisser.tif
	 Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/2.Dataset_res5/TPI/Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/HS/snowmap_2015_ads_100_extent_maskiert_nosnow0_noausreisser/extent.shp
Input shapes do not overlap raster.


/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


# TPIWGW

In [22]:
for filter_size in [11]: #[7, 11, 22, 50]:

    input_path_tpi = os.path.join(unitato_res, f"TPIWGW")
    output_path_tpi = os.path.join(unitato_base, f"TPIWGW")
    print("TPI:", filter_size)
    
    input_path_HS = f"{unitato_base}HS"
    for f in HS_files: 
        print(f)
        for tpi_file in os.listdir(input_path_tpi): 
            input_tpi = os.path.join(input_path_tpi, tpi_file)
            aoi = os.path.join(input_path_HS, os.path.splitext(f)[0], "extent.shp")
            try:
                cropped_im_data = access_raster(input_tpi, aoi=aoi)
            except ValueError as e: 
                print(e)
                continue
            print("\t", tpi_file)
            print(input_tpi)
            print(aoi)
            
            gdf = gpd.GeoDataFrame.from_file(aoi)
            rxr_tpi = rxr.open_rasterio(input_tpi, masked=True).squeeze()
            rxr_tpi = rxr_tpi.rio.clip(gdf.geometry.apply(mapping))
            
            try:
                os.makedirs(os.path.join(output_path_tpi, os.path.splitext(f)[0]))
            except: 
                pass
            rxr_tpi.rio.to_raster(os.path.join(output_path_tpi, os.path.splitext(f)[0], "TPIWGW.tif"))

TPI: 11
snowmap_20160309_ads_100_extent_maskiert_nosnow0_noausreisser.tif
	 Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/2.Dataset_res5/TPIWGW/Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/HS/snowmap_20160309_ads_100_extent_maskiert_nosnow0_noausreisser/extent.shp


/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


Input shapes do not overlap raster.
HS_Davos_20211604_2m_epsg2056.tif
	 Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/2.Dataset_res5/TPIWGW/Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/HS/HS_Davos_20211604_2m_epsg2056/extent.shp


/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


Input shapes do not overlap raster.
snowmap_2012_ads_80_extent_maskiert_nosnow0_noausreisser.tif
	 Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/2.Dataset_res5/TPIWGW/Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/HS/snowmap_2012_ads_80_extent_maskiert_nosnow0_noausreisser/extent.shp


...

/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


snowmap_20160126_ads_100_extent_maskiert_nosnow0_noausreisser.tif
	 Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/2.Dataset_res5/TPIWGW/Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/HS/snowmap_20160126_ads_100_extent_maskiert_nosnow0_noausreisser/extent.shp


/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


Input shapes do not overlap raster.
snowmap_2015_ads_100_extent_maskiert_nosnow0_noausreisser.tif
	 Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/2.Dataset_res5/TPIWGW/Davos.tif
/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/HS/snowmap_2015_ads_100_extent_maskiert_nosnow0_noausreisser/extent.shp
Input shapes do not overlap raster.


/home/abetatos/miniconda3/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


# DEMSNOW

In [25]:
import pandas as pd

lr_data_path = "/home/abetatos/Escritorio/TFM_Dataset/0.Dataset/stations_parsed.csv"
df = pd.read_csv(lr_data_path, sep=";")
# df

In [26]:
input_path_DEM =  os.path.join(unitato_base, "DEM")
output_path_DEM = os.path.join(unitato_base, "DEMSNOW")
os.makedirs(output_path_DEM, exist_ok=True)

input_path_HS = f"{unitato_base}HS"

for f in HS_files:
    print(f)
    input_DEM = os.path.join(input_path_DEM, os.path.splitext(f)[0], "DEM.tif")
    
    stat_data = df[df['place'] == f]
    a, b = stat_data[['a', 'b']].values[0]
    # y = a*x + b
    lf = lambda x: a*x + b if x != -9999. else x

    vf=np.vectorize(lf)
    
    with rio.open(input_DEM) as src: 
        profile = src.profile
        array = src.read()[0]
        new_array = vf(array)
    print(new_array.max(), new_array.min())
    out_path = os.path.join(output_path_DEM,  os.path.splitext(f)[0])
    try: 
        os.makedirs(out_path)
    except: 
        pass
    
    with rio.open(os.path.join(out_path, "DEMSNOW.tif"), "w", **profile) as src: 
        src.write(new_array, 1)
       

snowmap_20160309_ads_100_extent_maskiert_nosnow0_noausreisser.tif
2.708557719860658 0.5224479755698499
HS_Davos_20211604_2m_epsg2056.tif
2.4708472150709584 -0.022541299564496464
snowmap_2012_ads_80_extent_maskiert_nosnow0_noausreisser.tif
3.1188725073883505 0.4174293856903101
HS_Laucherenalp_20220224_2m_epsg2056.tif
4.476448185535514 0.3687903728081925
snowmap_2010_ads_80_extent_maskiert_nosnow0_noausreisser.tif
3.0977265781264967 -0.10230729737448452
HS_Davos_20200406_2m_epsg2056.tif
2.4708472150709584 -0.024398710641827304
HS_c_20220223_1m_epsg2056.tif
4.956120089426888 2.2373694374894297
snowmap_20160420_ads_100_extent_maskiert_nosnow0_noausreisser.tif
2.930796735999451 -0.1634094631823364
snowmap_2014_ads_100_extent_maskiert_nosnow0_noausreisser.tif
3.0548513035477916 -0.12247218941826277
HS_Davos_20180411_2m_epsg2056.tif
2.910072558425526 0.632270092555423
snowmap_2013_ads_80_extent_maskiert_nosnow0_noausreisser.tif
3.125921693610751 -0.08932707710336585
HS_Davos_20170316_2m_epsg2

# FFSNOW

far from snow

In [27]:
def lf(x): 
    if x == -9999.: 
        return x
    x = x - 240
    if x < -180: 
        x =  x + 360
    return abs(x) 

In [29]:
input_path_Aspect = os.path.join(unitato_base, "Aspect")
output_path_Aspect = os.path.join(unitato_base, "FFSnow")

input_path_HS = f"{unitato_base}HS"

for f in HS_files: 
    print(f)
   
    input_Aspect = os.path.join(input_path_Aspect, os.path.splitext(f)[0], "Aspect.tif")

    vf=np.vectorize(lf)
    
    with rio.open(input_Aspect) as src: 
        profile = src.profile
        array = src.read()[0]
        new_array = vf(array)
    print(new_array.max(), new_array.min())
    out_path = os.path.join(output_path_Aspect,  os.path.splitext(f)[0])
    try: 
        os.makedirs(out_path)
    except: 
        pass
    
    with rio.open(os.path.join(out_path, "FFSnow.tif"), "w", **profile) as src: 
        src.write(new_array, 1)

snowmap_20160309_ads_100_extent_maskiert_nosnow0_noausreisser.tif
179.99999618530273 0.0
HS_Davos_20211604_2m_epsg2056.tif
179.99999618530273 1.52587890625e-05
snowmap_2012_ads_80_extent_maskiert_nosnow0_noausreisser.tif
179.99999618530273 0.0
HS_Laucherenalp_20220224_2m_epsg2056.tif
179.9983901977539 0.000335693359375
snowmap_2010_ads_80_extent_maskiert_nosnow0_noausreisser.tif
179.99999618530273 0.0
HS_Davos_20200406_2m_epsg2056.tif
179.99999618530273 1.52587890625e-05
HS_c_20220223_1m_epsg2056.tif
179.99998092651367 0.001129150390625
snowmap_20160420_ads_100_extent_maskiert_nosnow0_noausreisser.tif
179.99999618530273 0.0
snowmap_2014_ads_100_extent_maskiert_nosnow0_noausreisser.tif
179.99999618530273 0.0
HS_Davos_20180411_2m_epsg2056.tif
179.99999237060547 1.52587890625e-05
snowmap_2013_ads_80_extent_maskiert_nosnow0_noausreisser.tif
179.99999618530273 0.0
HS_Davos_20170316_2m_epsg2056.tif
179.99999618530273 -9999.0
HS_Laucherenalp_20220511_1m_epsg2056.tif
179.9983901977539 0.000335